# ATMS 305: Module 8 Lecture 2
Mapping with `cartopy`
---

In [ ]:
%pylab inline
import xarray as xr
import pandas as pd
!pip install netcdf4  #install these if not installed
!pip install pydap

## Introduction to mapping with `cartopy`.

Python's `cartopy` is a package that is designed to enable `matplotlib` handle geographic calculations and visualize maps with geographic boundaries.  We can then use `matplotlib` to make maps, and off we go!

In [ ]:
#install cartopy if need be - should be default in ATMS 305 Azure Notebooks
!pip install cartopy

Cartopy has exposed an interface to enable easy map creation using matplotlib. Creating a basic map is as simple as telling matplotlib to use a specific map projection, and then adding some coastlines to the axes:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.show()

A list of the available projections to be used with `matplotlib` can be found on the Cartopy projection list page.

The line `plt.axes(projection=ccrs.PlateCarree())` sets up a `GeoAxes` instance which exposes a variety of other map related methods, in the case of the previous example, we used the `coastlines()` method to add coastlines to the map.

Lets create another map in a different projection, and make use of the `stock_img()` method to add an underlay image to the map:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.Mollweide())
ax.stock_img()
plt.show()

## Adding data to the map
Once you have the map just the way you want it, data can be added to it in exactly the same way as with normal matplotlib axes. By default, the coordinate system of any data added to a GeoAxes is the same as the coordinate system of the GeoAxes itself, to control which coordinate system that the given data is in, you can add the `transform` keyword with an appropriate `cartopy.crs.CRS` instance:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()

ny_lon, ny_lat = -75, 43
delhi_lon, delhi_lat = 77.23, 28.61

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='blue', linewidth=2, marker='o',
         transform=ccrs.Geodetic(),
         )

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='gray', linestyle='--',
         transform=ccrs.PlateCarree(),
         )

plt.text(ny_lon - 3, ny_lat - 12, 'New York',
         horizontalalignment='right',
         transform=ccrs.Geodetic())

plt.text(delhi_lon + 3, delhi_lat - 12, 'Delhi',
         horizontalalignment='left',
         transform=ccrs.Geodetic())

plt.show()

# More advanced mapping with `cartopy` and `matplotlib`

From the outset, `cartopy`’s purpose has been to simplify and improve the quality of mapping visualisations available for scientific data. Thanks to the simplicity of the cartopy interface, in many cases the hardest part of producing such visualisations is getting hold of the data in the first place. 

In [ ]:
import os
import matplotlib.pyplot as plt

import numpy as np

from cartopy import config
import cartopy.crs as ccrs


# get the path of the file. It can be found in the repo data directory.
sst_data = xr.open_dataset('https://www.ncei.noaa.gov/thredds/dodsC/ncFC/fc-oisst-daily-avhrr-only-dly/OISST_Daily_AVHRR-only_Feature_Collection_best.ncd').sel(time=pd.datetime(2017,8,1))

sst_data

In [ ]:
plt.figure(figsize=(10,5))

ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(sst_data['lon'], sst_data['lat'], sst_data['sst'].values.squeeze(), 20,
             transform=ccrs.PlateCarree())

plt.title(sst_data.title+'\n'+str(sst_data['time'].values))
ax.coastlines()

plt.colorbar()
plt.tight_layout()
plt.show()

## Vector plotting

Cartopy comes with powerful vector field plotting functionality. There are 3 distinct options for visualising vector fields: quivers, barbs, and streamplots, each with their own benefits for displaying certain vector field forms.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import cartopy
import cartopy.crs as ccrs


def sample_data(shape=(20, 30)):
    """
    Returns ``(x, y, u, v, crs)`` of some vector data
    computed mathematically. The returned crs will be a rotated
    pole CRS, meaning that the vectors will be unevenly spaced in
    regular PlateCarree space.

    """
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)

    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])

    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))

    return x, y, u, v, crs

plt.figure(figsize=(10,10))

ax = plt.axes(projection=ccrs.Orthographic(-10, 45))

ax.add_feature(cartopy.feature.OCEAN, zorder=0)
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')

ax.set_global()
ax.gridlines()

x, y, u, v, vector_crs = sample_data()
ax.quiver(x, y, u, v, transform=vector_crs)

plt.show()

## Putting it back together

Now, let's work with our global temperature anomaly dataset.  We will use xarray to select some times, and visualize the anomalies on a two panel plot.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.ticker as mticker

In [ ]:
nc=xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc')
nc

In [ ]:
#Select January 2017, make lats and lons 2D so basemap knows where to plot each point

jan1976 = nc.sel(time=pd.datetime(1976,1,1))
jan2018 = nc.sel(time=pd.datetime(2018,1,1))

Here, let's plot the anomalies from these two time periods on a rectangular map projection called "PlateCarree" in a two panel plot.

In [ ]:
# select data we want
data = jan1976['air']
data2 = jan2018['air']

# create figure, axes instances.
fig = plt.figure(figsize=(12, 3))

ax = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
data.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax.set_global(); 
ax.coastlines();
#set up the gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')
#set where the gridlines go
gl.xlocator = mticker.FixedLocator(np.arange(-180,181,15))
gl.ylocator = mticker.FixedLocator(np.arange(-90,91,15))

ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
data2.plot.contourf(ax=ax2, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax2.set_global(); 
ax2.coastlines();

gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1, color='gray', alpha=0.5, linestyle='-')

gl2.ylocator = mticker.FixedLocator(np.arange(-90,91,15))
gl2.xlocator = mticker.FixedLocator(np.arange(-180,181,15))

plt.tight_layout()
plt.show()

Let's zoom in to the Eastern US - and add some fancy stuff like country and state borders, and lakes.

In [ ]:
import cartopy.feature as cfeature

# select data we want
data = jan1976['air']
data2 = jan2018['air']

# create figure, axes instances.
fig = plt.figure(figsize=(12, 6))

cartopy_proj = projection=ccrs.Mercator()

ax = plt.subplot(1, 2, 1, projection=cartopy_proj)
data.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax.set_extent([-100,-80,30.,50.], crs=ccrs.PlateCarree()) 
ax.coastlines();
ax.add_feature(cfeature.STATES, edgecolor='gray')
ax.add_feature(cfeature.LAKES, edgecolor='gray')
ax.add_feature(cfeature.BORDERS, edgecolor='gray')
#set up the gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=2, color='gray', alpha=0.5, linestyle='-')
#set where the gridlines go
gl.xlocator = mticker.FixedLocator(np.arange(-180,181,5))
gl.ylocator = mticker.FixedLocator(np.arange(-90,90,5))

ax2 = plt.subplot(1, 2, 2, projection=cartopy_proj)
data2.plot.contourf(ax=ax2, transform=ccrs.PlateCarree(), levels=np.linspace(-12, 12, 25));
ax2.set_extent([-100,-80,30.,50.], crs=ccrs.PlateCarree()) 
ax2.coastlines();
ax2.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='gray') 
ax2.add_feature(cfeature.LAKES, edgecolor='gray')
ax2.add_feature(cfeature.BORDERS, edgecolor='gray')
gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=2, color='gray', alpha=0.5, linestyle='-')

gl2.ylocator = mticker.FixedLocator(np.arange(-90,90,5))
gl2.xlocator = mticker.FixedLocator(np.arange(-180,181,5))

plt.tight_layout()
plt.show()